In [ ]:
import os
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn as nn
from torchsummary import summary
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [54]:
import cv2
import os
import numpy as np

def rotate_image(image, angle):
    """Rotate the image by a certain angle."""
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def augment_and_save_images_to_100(input_folder, output_folder, current_count, target_count=100):
    """Augment images by rotating to reach a target count, save them and the original images to the output folder."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    files = [f for f in os.listdir(input_folder) if f.endswith(".jpg") or f.endswith(".png")]
    num_original_images = len(files)
    num_required_images = target_count - current_count

    if num_required_images <= 0:
        print(f"No augmentation needed for {input_folder}.")
        return

    rotations_per_image = (num_required_images + num_original_images - 1) // num_original_images  # Round up
    rotation_angle = 360 / rotations_per_image

    current_count = 0

    for filename in files:
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Save the original image to the augmented folder
        original_output_path = os.path.join(output_folder, filename)
        cv2.imwrite(original_output_path, image)
        current_count += 1

        for i in range(1, rotations_per_image):
            if current_count >= target_count:
                break
            angle = i * rotation_angle
            rotated_image = rotate_image(image, angle)
            base_filename = os.path.splitext(filename)[0]
            output_path = os.path.join(output_folder, f"{base_filename}_rot_{int(angle)}.jpg")
            cv2.imwrite(output_path, rotated_image)
            current_count += 1
            if current_count >= target_count:
                break

# Define folders
sad_folder = "/content/drive/My Drive/cats facial expressions without aug/Sad"
happy_folder = "/content/drive/My Drive/cats facial expressions without aug/happy"
angry_folder = "/content/drive/My Drive/cats facial expressions without aug/Angry"

augmented_sad_folder = "/content/drive/My Drive/cats facial expressions without aug/Sad_augmented"
augmented_happy_folder = "/content/drive/My Drive/cats facial expressions without aug/Happy_augmented"
augmented_angry_folder = "/content/drive/My Drive/cats facial expressions without aug/Angry_augmented"

# Original counts
original_counts = {
    'Sad': 14,
    'Happy': 8,
    'Angry': 46
}

# Augment each class to have about 100 unique images
augment_and_save_images_to_100(sad_folder, augmented_sad_folder, original_counts['Sad'], target_count=100)
augment_and_save_images_to_100(happy_folder, augmented_happy_folder, original_counts['Happy'], target_count=100)
augment_and_save_images_to_100(angry_folder, augmented_angry_folder, original_counts['Angry'], target_count=100)

print("Augmentation to reach about 100 unique images per class completed.")


Augmentation to reach about 100 unique images per class completed.


In [55]:
def count_images_in_folders(folders):
    counts = {}
    for folder in folders:
        counts[folder] = len([name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))])
    return counts


folders = [
    "/content/drive/My Drive/cats facial expressions without aug/Sad",
    "/content/drive/My Drive/cats facial expressions without aug/Angry",
    "/content/drive/My Drive/cats facial expressions without aug/happy",
    "/content/drive/My Drive/cats facial expressions without aug/Sad_augmented",
    "/content/drive/My Drive/cats facial expressions without aug/Happy_augmented",
     "/content/drive/My Drive/cats facial expressions without aug/Angry_augmented"
]

# Count images in each folder
image_counts = count_images_in_folders(folders)

# Print the counts
for folder, count in image_counts.items():
    print(f"{folder}: {count} images")

/content/drive/My Drive/cats facial expressions without aug/Sad: 14 images
/content/drive/My Drive/cats facial expressions without aug/Angry: 46 images
/content/drive/My Drive/cats facial expressions without aug/happy: 8 images
/content/drive/My Drive/cats facial expressions without aug/Sad_augmented: 98 images
/content/drive/My Drive/cats facial expressions without aug/Happy_augmented: 96 images
/content/drive/My Drive/cats facial expressions without aug/Angry_augmented: 92 images
